In [13]:
from pprint import pprint
from lxml import html
import requests
import re
import datetime
import pandas as pd

header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'}

result = []
def lenta_news(url):
    try:
        # result = []
        main_url = 'https://lenta.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)

        names = root.xpath("//div[@class = 'span8 js-main__content']//div[@class='item']")
        for el in names:
            items = {}
            url = el.xpath(".//a[@href]/@href")
            title = el.xpath(".//a[@href]/text()")
            date = el.xpath(".//a[@href]/time/@title")
            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            items['date'] = date[0]
            items['sourse'] = 'lenta.ru'
            result.append(items)

        return result
    except:
        print('Ошибка запроса')

def yandex_news(url):
    try:
        # result = []
        main_url = 'https://yandex.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)
        topics_info = root.xpath("//td[@class = 'stories-set__item']")

        for el in topics_info:
            items = {}
            url = el.xpath(".//a[contains(@class, 'link_theme_black')]/@href")
            title = el.xpath(".//a[contains(@class, 'link_theme_black')]/text()")
            date_full = el.xpath(".//div[@class = 'story__date']/text()")[0].replace(u'\xa0', ' ')
            date = el.xpath(".//div[@class = 'story__date']/text()")

            if re.search('вчера', date_full):
                date = str((datetime.date.today() - datetime.timedelta(days=1)))+ " " + date[0].replace(u'\xa0', ' ')[-5:]
                source = date_full[:-14]
            else:
                date = str(datetime.date.today())+ " " + date[0].replace(u'\xa0', ' ')[-5:]
                source = date_full[:-6]

            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            # items['date_full'] = date_full
            items['date'] = date
            items['source'] = source
            result.append(items)
        return result
    except:
        print('Ошибка запроса')

def mail_news(url):
    try:
        # result = []
        main_url = 'https://news.mail.ru'
        response = requests.get(url, headers=header)
        root = html.fromstring(response.text)
        topics_info = root.xpath("//div[@class = 'js-module']//li[contains(@class, 'list__item')]//a[@class = 'list__text']")

        for el in topics_info:
            items = {}
            url = el.xpath("./@href")
            for item in url:
                if re.search('https', item):
                    deep_response = requests.get(item, headers=header)
                    deep_root = html.fromstring(deep_response.text)
                    source = deep_root.xpath(".//a[@class = 'link color_gray breadcrumbs__link']//text()")
                    date = deep_root.xpath("//span[@class = 'note__text breadcrumbs__text js-ago'] / @ datetime")
                else:
                    deep_response = requests.get(main_url + item, headers=header)
                    deep_root = html.fromstring(deep_response.text)
                    source = deep_root.xpath(".//a[@class = 'link color_gray breadcrumbs__link']//text()")
                    date = deep_root.xpath("//span[@class = 'note__text breadcrumbs__text js-ago'] / @ datetime")
            title = el.xpath("./text()")
            items['url'] = main_url + url[0]
            items['title'] = title[0].replace(u'\xa0', ' ')
            items['date'] = date[0]
            items['source'] = source[0]
            result.append(items)
        return result
    except:
        print('Ошибка запроса')

lenta_news = lenta_news('https://lenta.ru/')
yandex_news = yandex_news('https://yandex.ru/news/')
mail_news = mail_news('https://news.mail.ru/')

# pprint((mail_news))




from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['news_db']
news = db.news

news.insert_many(mail_news)

In [63]:
news = db.news #selecting the coll1 in myDatabase
for document in news.find():
    print(document)


{'_id': ObjectId('5e4f9215451addc48b28a625'), 'url': 'https://lenta.ru/news/2020/02/21/intruder/', 'title': '102-летний пенсионер отбился от ворвавшегося в дом грабителя', 'date': '21 февраля 2020', 'sourse': 'lenta.ru'}
{'_id': ObjectId('5e4f9215451addc48b28a626'), 'url': 'https://lenta.ru/news/2020/02/21/loot/', 'title': 'Путин обвинил бывшие власти Украины в стремлении «сохранить награбленное»', 'date': '21 февраля 2020', 'sourse': 'lenta.ru'}
{'_id': ObjectId('5e4f9215451addc48b28a627'), 'url': 'https://lenta.ru/news/2020/02/21/stihi/', 'title': 'Собчак сравнила Шнурова в политике с лошадью', 'date': '21 февраля 2020', 'sourse': 'lenta.ru'}
{'_id': ObjectId('5e4f9215451addc48b28a628'), 'url': 'https://lenta.ru/news/2020/02/21/generosity_hurts/', 'title': 'Москвичка отдала незнакомцу миллион рублей на покупку жилья', 'date': '21 февраля 2020', 'sourse': 'lenta.ru'}
{'_id': ObjectId('5e4f9215451addc48b28a629'), 'url': 'https://lenta.ru/news/2020/02/21/accord/', 'title': 'Путин расска

In [56]:
import pandas as pd
import json
import bson
from bson import Binary, Code
from bson.json_util import dumps

In [107]:
news = db.news #selecting the coll1 in myDatabase
filter = {"type": "something"}

documents_count = db['news'].estimated_document_count()
print(documents_count)

with open("collection.json", "w") as file:
    file.write('[')

    for i, document in enumerate(news.find(), 1):
        file.write(json.dumps(document, default = str))
        if i != documents_count:
            file.write(',')
    file.write(']')

    
    
# filter = {"type": "something"}
# type_documents = db['cluster'].find(filter)
# type_documents_count = db['cluster'].count_documents(filter)

# with open("type_documents.json", "w") as file:
#     file.write('[')
#     # Start from one as type_documents_count also starts from 1.
#     for i, document in enumerate(type_documents, 1):
#         file.write(json.dumps(document, default=str))
#         if i != type_documents_count:
#             file.write(',')
#     file.write(']')
    

81


In [108]:
news_pd = pd.read_json(r'collection.json')
news_pd

,_id,date,source,sourse,title,url
0,5e4f9215451addc48b28a625,21 февраля 2020,NaN,lenta.ru,102-летний пенсионер отбился от ворвавшегося в...,https://lenta.ru/news/2020/02/21/intruder/
1,5e4f9215451addc48b28a626,21 февраля 2020,NaN,lenta.ru,Путин обвинил бывшие власти Украины в стремлен...,https://lenta.ru/news/2020/02/21/loot/
2,5e4f9215451addc48b28a627,21 февраля 2020,NaN,lenta.ru,Собчак сравнила Шнурова в политике с лошадью,https://lenta.ru/news/2020/02/21/stihi/
3,5e4f9215451addc48b28a628,21 февраля 2020,NaN,lenta.ru,Москвичка отдала незнакомцу миллион рублей на ...,https://lenta.ru/news/2020/02/21/generosity_hu...
4,5e4f9215451addc48b28a629,21 февраля 2020,NaN,lenta.ru,Путин рассказал о надежде на договоренность с ...,https://lenta.ru/news/2020/02/21/accord/
5,5e4f9215451addc48b28a62a,21 февраля 2020,NaN,lenta.ru,Российский заключенный сбежал из колонии после...,https://lenta.ru/news/2020/02/21/collectorrunner/
6,5e4f9215451addc48b28a62b,21 февраля 2020,NaN,lenta.ru,Футболист спародировал «страдания» соперника-с...,https://lenta.ru/news/2020/02/21/babel/
7,5e4f9215451addc48b28a62c,21 февраля 2020,NaN,lenta.ru,Путин заявил о страхе Европы перед объединенно...,https://lenta.ru/news/2020/02/21/ru_ukr/
8,5e4f9215451addc48b28a62d,21 февраля 2020,NaN,lenta.ru,В «проклятой» египетской гробнице совершили ре...,https://lenta.ru/news/2020/02/21/nefertiti/
9,5e4f9215451addc48b28a62e,2020-02-21 12:32,Российская газета,NaN,Анкара исключила военный конфликт с Россией в ...,https://yandex.ru/news/story/Ankara_isklyuchil...
